In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        beta = 0.05
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * beta)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=19)
edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=70)
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
accel = ContinuousParameter('accel', 1.5, 5.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, edgeLength, numberOfLanes, accel])
#parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/520_init_points_timeLoss.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 520
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_timeLoss.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

[[12.         15.18537634 50.07695676  3.          3.23591845]
 [ 9.         17.64316124 33.07155348  1.          1.69009664]
 [ 3.         13.86571269 63.46741532  1.          2.87678704]
 ...
 [ 6.         17.59296285 64.48763277  1.          1.63476688]
 [20.         18.53450361 56.24677653  2.          4.61300791]
 [ 9.         17.66143035 68.83997072  2.          2.28589339]]

User function time loss called with 520 inputs to simulate

Evaluating input: 1 of 520

 Retrying in 1 seconds

Output 0.32166762287244216


Evaluating input: 2 of 520

 Retrying in 1 seconds

Output 0.4984751871361242


Evaluating input: 3 of 520

 Retrying in 1 seconds

Output 0.2962962962962963


Evaluating input: 4 of 520

 Retrying in 1 seconds

Output 0.3457230142566191


Evaluating input: 5 of 520

 Retrying in 1 seconds

Output 0.25359911406423036


Evaluating input: 6 of 520

 Retrying in 1 seconds

Output 0.48180324019722937


Evaluating input: 7 of 520

 Retrying in 1 seconds

Output 0.49679117985


Output 0.3189998407389712


Evaluating input: 97 of 520

 Retrying in 1 seconds

Output 0.35820895522388063


Evaluating input: 98 of 520

 Retrying in 1 seconds

Output 0.265818858560794


Evaluating input: 99 of 520

 Retrying in 1 seconds

Output 0.17953144266337856


Evaluating input: 100 of 520

 Retrying in 1 seconds

Output 0.39834668044432964


Evaluating input: 101 of 520

 Retrying in 1 seconds

Output 0.3062815222375057


Evaluating input: 102 of 520

 Retrying in 1 seconds

Output 0.4742599742599743


Evaluating input: 103 of 520

 Retrying in 1 seconds

Output 0.3447566826593557


Evaluating input: 104 of 520

 Retrying in 1 seconds

Output 0.24611801242236023


Evaluating input: 105 of 520

 Retrying in 1 seconds

Output 0.18195970695970695


Evaluating input: 106 of 520

 Retrying in 1 seconds

Output 0.1977671451355662


Evaluating input: 107 of 520

 Retrying in 1 seconds

Output 0.20466840312562615


Evaluating input: 108 of 520

 Retrying in 1 seconds

Output 0.3506

 Retrying in 1 seconds

Output 0.1606053209665609


Evaluating input: 197 of 520

 Retrying in 1 seconds

Output 0.4336442973216973


Evaluating input: 198 of 520

 Retrying in 1 seconds

Output 0.2701338342333881


Evaluating input: 199 of 520

 Retrying in 1 seconds

Output 0.3438533978111479


Evaluating input: 200 of 520

 Retrying in 1 seconds

Output 0.3202020202020202


Evaluating input: 201 of 520

 Retrying in 1 seconds

Output 0.32305220883534136


Evaluating input: 202 of 520

 Retrying in 1 seconds

Output 0.1985989102635383


Evaluating input: 203 of 520

 Retrying in 1 seconds

Output 0.3979825767996332


Evaluating input: 204 of 520

 Retrying in 1 seconds

Output 0.3971770241129191


Evaluating input: 205 of 520

 Retrying in 1 seconds

Output 0.2667538886466056


Evaluating input: 206 of 520

 Retrying in 1 seconds

Output 0.22951414068165338


Evaluating input: 207 of 520

 Retrying in 1 seconds

Output 0.3794832240647898


Evaluating input: 208 of 520

 Retrying in 1

 Retrying in 1 seconds

Output 0.21567483469467133


Evaluating input: 297 of 520

 Retrying in 1 seconds

Output 0.3151680290644868


Evaluating input: 298 of 520

 Retrying in 1 seconds

Output 0.32689987937273823


Evaluating input: 299 of 520

 Retrying in 1 seconds

Output 0.3391326273323248


Evaluating input: 300 of 520

 Retrying in 1 seconds

Output 0.36528877162122547


Evaluating input: 301 of 520

 Retrying in 1 seconds

Output 0.15878599476439792


Evaluating input: 302 of 520

 Retrying in 1 seconds

Output 0.2510930668332292


Evaluating input: 303 of 520

 Retrying in 1 seconds

Output 0.30852842809364545


Evaluating input: 304 of 520

 Retrying in 1 seconds

Output 0.5247619047619048


Evaluating input: 305 of 520

 Retrying in 1 seconds

Output 0.47721280602636534


Evaluating input: 306 of 520

 Retrying in 1 seconds

Output 0.5387387387387388


Evaluating input: 307 of 520

 Retrying in 1 seconds

Output 0.37049424005945747


Evaluating input: 308 of 520

 Retrying


Output 0.3819951338199513


Evaluating input: 396 of 520

 Retrying in 1 seconds

Output 0.25497737556561084


Evaluating input: 397 of 520

 Retrying in 1 seconds

Output 0.2691537761601456


Evaluating input: 398 of 520

 Retrying in 1 seconds

Output 0.23196022727272722


Evaluating input: 399 of 520

 Retrying in 1 seconds

Output 0.4436919504643963


Evaluating input: 400 of 520

 Retrying in 1 seconds

Output 0.2346825923468259


Evaluating input: 401 of 520

 Retrying in 1 seconds

Output 0.40760644894584536


Evaluating input: 402 of 520

 Retrying in 1 seconds

Output 0.26058252427184464


Evaluating input: 403 of 520

 Retrying in 1 seconds

Output 0.20579268292682928


Evaluating input: 404 of 520

 Retrying in 1 seconds

Output 0.2996742671009772


Evaluating input: 405 of 520

 Retrying in 1 seconds

Output 0.4292307692307692


Evaluating input: 406 of 520

 Retrying in 1 seconds

Output 0.3010212623472292


Evaluating input: 407 of 520

 Retrying in 1 seconds

Output 0.3

 Retrying in 1 seconds

Output 0.22117587172308475


Evaluating input: 496 of 520

 Retrying in 1 seconds

Output 0.2136627906976744


Evaluating input: 497 of 520

 Retrying in 1 seconds

Output 0.3563155989866088


Evaluating input: 498 of 520

 Retrying in 1 seconds

Output 0.20912240184757505


Evaluating input: 499 of 520

 Retrying in 1 seconds

Output 0.24778463264161524


Evaluating input: 500 of 520

 Retrying in 1 seconds

Output 0.24785426519530565


Evaluating input: 501 of 520

 Retrying in 1 seconds

Output 0.20667226185480486


Evaluating input: 502 of 520

 Retrying in 1 seconds

Output 0.20407747196738024


Evaluating input: 503 of 520

 Retrying in 1 seconds

Output 0.2625338753387534


Evaluating input: 504 of 520

 Retrying in 1 seconds

Output 0.3730696288268762


Evaluating input: 505 of 520

 Retrying in 1 seconds

Output 0.20475075276012045


Evaluating input: 506 of 520

 Retrying in 1 seconds

Output 0.32043448744059744


Evaluating input: 507 of 520

 Retryin

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = -760.9859065400349
Optimization restart 2/5, f = -1332.8666251309153
Optimization restart 3/5, f = -1332.8666251320203
Optimization restart 4/5, f = -1332.8666251262825
Optimization restart 5/5, f = -1332.8666250579658


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design/models/5param_model_variance_20_init_points_500_loops_timeloss_per_duration.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/5param_integrated_variance_reduction_20_init_points_500_loops_timeloss_per_duration.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -1332.8666251366155
Optimization restart 2/5, f = -1332.8666251281352
Optimization restart 3/5, f = -1332.866625110766
Optimization restart 4/5, f = -1332.8666250141428
Optimization restart 5/5, f = -1332.866625102475
[[ 9.          8.         70.          1.          2.14174684]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1277438334464811

Optimization restart 1/5, f = -1335.9830146079219
Optimization restart 2/5, f = -1335.9830377542928
Optimization restart 3/5, f = -1335.9830378960924
Optimization restart 4/5, f = -1335.983037899824
Optimization restart 5/5, f = -1335.9830378951115
[[10.          8.         70.          1.          2.13355498]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1296392556398794

Optimization restart 1/5, f = -1339.1407165314024
Optimization restart 2/5, f = -1339.1407424260142
Opt

Optimization restart 4/5, f = -1388.12249774273
Optimization restart 5/5, f = -1388.1224977478737
[[10.          8.         70.          1.          2.25355026]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13006440889258258

Optimization restart 1/5, f = -1391.4388526190612
Optimization restart 2/5, f = -1391.4389118129884
Optimization restart 3/5, f = -1391.4389117901449
Optimization restart 4/5, f = -1391.4389117599699
Optimization restart 5/5, f = -1391.4389118315253
[[10.          8.         70.          1.          2.25413043]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13006440889258258

Optimization restart 1/5, f = -1394.7590259545054
Optimization restart 2/5, f = -1394.7590858844774
Optimization restart 3/5, f = -1394.7590858639624
Optimization restart 4/5, f = -1394.7590858868825
Optimization restart 5/5, f = -1394.7590858745102


 Retrying in 1 seconds

Output 0.13070937182812675

Optimization restart 1/5, f = -1451.4172198901235
Optimization restart 2/5, f = -1451.4172830169737
Optimization restart 3/5, f = -1451.4172829557504
Optimization restart 4/5, f = -1451.4172830103182
Optimization restart 5/5, f = -1451.4172830211055
[[10.          8.         70.          1.          2.24583117]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1307596513075965

Optimization restart 1/5, f = -1454.7772136238877
Optimization restart 2/5, f = -943.3062338645403
Optimization restart 3/5, f = -943.3062338931268
Optimization restart 4/5, f = -1454.7772766287899
Optimization restart 5/5, f = -1454.7772766284797
[[ 9.          8.         70.          1.          2.24551308]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12896668548842463

Optimization restart 1/5, f = -1458.1326365005143

In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -1339.221653862728
Optimization restart 2/5, f = -1339.221653860982
Optimization restart 3/5, f = -1339.2216538836315
Optimization restart 4/5, f = -1339.2216538807534
Optimization restart 5/5, f = -1339.2216538533794
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1342.6504216369726
Optimization restart 2/5, f = -1342.6504439483103
Optimization restart 3/5, f = -1342.650450485512
Optimization restart 4/5, f = -1342.65045049335
Optimization restart 5/5, f = -1342.6504504944492
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1346.0987219641756
Optimization restart 2/5, f = -1346.0987596844525
Optimization restart 3/5, f = -1346.098759495494
Optimization restart 4/5, 

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1408.8700138069673
Optimization restart 2/5, f = -1408.870070490723
Optimization restart 3/5, f = -1408.8700706435095
Optimization restart 4/5, f = -1408.8700688810059
Optimization restart 5/5, f = -1408.87007070374
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1412.4097576395252
Optimization restart 2/5, f = -1412.4098144696509
Optimization restart 3/5, f = -1412.4098143990427
Optimization restart 4/5, f = -1412.4098145007326
Optimization restart 5/5, f = -1412.409814527885
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1415.9516944675734
Optimization restart 2/5, f = -1415.951751007674
Optimization restart 3/5

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1480.0271347767134
Optimization restart 2/5, f = -1480.0271866509502
Optimization restart 3/5, f = -1126.6740843858668
Optimization restart 4/5, f = -1480.0271871155003
Optimization restart 5/5, f = -1480.0271871310465
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1483.6027623207085
Optimization restart 2/5, f = -1483.6028142883222
Optimization restart 3/5, f = -1483.602812898421
Optimization restart 4/5, f = -1483.6028142522673
Optimization restart 5/5, f = -1483.6028141274996
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -1487.1799258201213
Optimization restart 2/5, f = -1487.1799776024882
Optimization restart

In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -1371.5809591134262
Optimization restart 2/5, f = -1371.5809591259447
Optimization restart 3/5, f = -1371.5809590835556
Optimization restart 4/5, f = -1371.5809591090333
Optimization restart 5/5, f = -1371.5809591049797
[[11.         8.        70.         2.         1.8187111]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13397082018927445

Optimization restart 1/5, f = -1375.0042734082735
Optimization restart 2/5, f = -1375.0042755912912
Optimization restart 3/5, f = -1375.004275578739
Optimization restart 4/5, f = -1375.0042755588638
Optimization restart 5/5, f = -1375.0042752171732
[[20.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13209992929530992

Optimization restart 1/5, f = -1378.1160878154783
Optimization restart 2/5, f = -1378.116136779082
Optimization restart 3/5, f = -1378.1161

 Retrying in 1 seconds

Output 0.13209992929530992

Optimization restart 1/5, f = -1436.3979713760345
Optimization restart 2/5, f = -1436.3979943779234
Optimization restart 3/5, f = -1436.3979964899456
Optimization restart 4/5, f = -1436.3979964571095
Optimization restart 5/5, f = -1436.397996443492
[[10.          8.         70.          2.          1.90071478]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.129704875962361

Optimization restart 1/5, f = -1439.908642984767
Optimization restart 2/5, f = -1439.9086809008738
Optimization restart 3/5, f = -1439.9086808914692
Optimization restart 4/5, f = -1439.908580749241
Optimization restart 5/5, f = -1439.9086809016037
[[10.          8.         70.          2.          1.90000343]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1298770710849813

Optimization restart 1/5, f = -1443.4282188900963
Op

Optimization restart 4/5, f = -1499.1083727060573
Optimization restart 5/5, f = -1499.1083729982565
[[11.          8.         62.11269821  1.          1.9697345 ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13571585178688886

Optimization restart 1/5, f = -1502.6618485370761
Optimization restart 2/5, f = -1502.6618790975654
Optimization restart 3/5, f = -1502.6618791325336
Optimization restart 4/5, f = -1502.6618791072935
Optimization restart 5/5, f = -1502.6618789513032
[[11.          8.         62.52005227  1.          1.96134906]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1345150065245759

Optimization restart 1/5, f = -1506.2123508802804
Optimization restart 2/5, f = -1506.2123721889998
Optimization restart 3/5, f = -1506.2123721673004
Optimization restart 4/5, f = -1506.2123721597163
Optimization restart 5/5, f = -1506.2123720692666

## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
results_random_init = bo_random_init.get_results()

In [19]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,9.0,8.0,70.0,1.0,2.174546


In [20]:
results_random_init.minimum_value*100

12.761624618169476

### Results on the experimentally designed emulator with model variance

In [21]:
results_model_variance = bo_model_variance.get_results()

In [22]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,9.0,8.0,70.0,1.0,1.5


In [23]:
results_model_variance.minimum_value*100

12.286689419795222

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,10.0,8.0,70.0,2.0,1.862688


In [26]:
results_integrated_variance.minimum_value*100

12.376237623762377

## Save stuff

In [27]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_timeLoss.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
with open(f'bayesian_optimization/models/520_random_init_50_bay_opt_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization/models/520_model_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/520_integrated_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)